# 출력 파서 (`OutputParser`)
- LLM의 출력을 더 유용/구조화된 형태로 변환함
- 장점
    1. 구조화: LLM의 자유 형식 출력을 구조화된 데이터로 변환할 수 있음 (json, dict)
    1. 일관성: 항상 일관된 출력 형식 -> 후속 처리에 용이하다
    1. 유연성: 다양한 출력 형식(JSON, list, dict)으로 변환 가능

## `PydanticOutputParser`

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model='gpt-4.1-nano')

In [3]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [8]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """아래 이메일 내용 중 중요한 것만 추출해.
---
{email_conversation}
"""
)

print(prompt.format(email_conversation=email_conversation)) # 각각 빵꾸, 변수명

chain = prompt | llm
chain.invoke({'email_conversation': email_conversation}).content

아래 이메일 내용 중 중요한 것만 추출해.
---
From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션




'[중요 내용 요약]\n\n- 발신자: 김철수 상무, 바이크코퍼레이션\n- 수신자: 이은채 대리님, 테디인터내셔널\n- 목적:\n  - "ZENESIS" 자전거 유통 협력 문의\n  - 상세 브로슈어 요청 (기술 사양, 배터리 성능, 디자인)\n  - 협력 가능성 논의를 위한 미팅 제안\n- 미팅 일정: 1월 15일 화요일 오전 10시 (귀사 사무실에서)'

In [9]:
print('[중요 내용 요약]\n\n- 발신자: 김철수 상무, 바이크코퍼레이션\n- 수신자: 이은채 대리님, 테디인터내셔널\n- 목적:\n  - "ZENESIS" 자전거 유통 협력 문의\n  - 상세 브로슈어 요청 (기술 사양, 배터리 성능, 디자인)\n  - 협력 가능성 논의를 위한 미팅 제안\n- 미팅 일정: 1월 15일 화요일 오전 10시 (귀사 사무실에서)')

[중요 내용 요약]

- 발신자: 김철수 상무, 바이크코퍼레이션
- 수신자: 이은채 대리님, 테디인터내셔널
- 목적:
  - "ZENESIS" 자전거 유통 협력 문의
  - 상세 브로슈어 요청 (기술 사양, 배터리 성능, 디자인)
  - 협력 가능성 논의를 위한 미팅 제안
- 미팅 일정: 1월 15일 화요일 오전 10시 (귀사 사무실에서)


In [17]:
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='보낸 사람의 메일 주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문 요약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')

parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸 사람의 메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [21]:
prompt = PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴스야. 아래 질문에 맞게 답변을 한국어로 만들어줘

질문: {question}
이메일 내용: {email_conversation}
형식 {format}
"""
)

# 프롬프트 변수들 중 일부만 채우기
prompt = prompt.partial(format = parser.get_format_instructions())

chain = prompt | llm | parser

res = chain.invoke(
    {'question': '이메일 내용 중 중요한 내용 추출해줘',
     'email_conversation': email_conversation}
)


from pprint import pprint

pprint(res.model_dump())

{'date': '1월 15일 오전 10시',
 'email': 'chulsoo.kim@bikecorporation.me',
 'person': '김철수',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': "김철수 상무는 귀사의 'ZENESIS' 자전거에 대해 상세 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 "
            '등에 관한 정보를 원함. 또한, 1월 15일 오전 10시에 사무실에서 미팅을 제안하여 협력 가능성을 논의하고자 함.'}


## CSV Parser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# CSV 파서의 안내사항 확인
print(output_parser.get_format_instructions())


prompt = PromptTemplate(
    template='List 5 {subject}\n{format_instructions}',
    input_variables=['subject'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

print('---')
print(prompt.format(subject='중국집 대표메뉴'))

chain1 = prompt | llm
csv_res = chain1.invoke({'subject': '맥도날드 대표메뉴'}).content


chain2 = prompt | llm | output_parser
list_res = chain2.invoke({'subject': '국민주식'})


Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
---
List 5 중국집 대표메뉴
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
Big Mac, Quarter Pounder, Chicken McNuggets, McToast, Fries
['삼성전자', 'SK하이닉스', '현대차', 'LG화학', 'NCSOFT']


In [ ]:
print(csv_res)
print(list_res) # 리스트가 더 활용도 좋음

Big Mac, Quarter Pounder, Chicken McNuggets, McToast, Fries
['삼성전자', 'SK하이닉스', '현대차', 'LG화학', 'NCSOFT']


## Structured Output Parser
- dict 형식 -> 멍청한 모델에도 적용 가능

In [31]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schema = [
    ResponseSchema(name='answer', description='사용자 질문에 대한 답변'),
    ResponseSchema(name='source', description='질문에 답하기 위해 사용된 출처')
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [36]:
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='사용자 질문에 최선을 다해 답변한다. \n{format_instructions}\n{question}',
    input_variables=['question'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

print(prompt.format(question='처서는 몇월 며칠인가?'))

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 출처
}
```
사용자 질문에 최선을 다해 답변한다. 
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 출처
}
```
처서는 몇월 며칠인가?


In [37]:
chain = prompt|llm|output_parser
chain.invoke({'question':'처서는 언제야?'})

{'answer': '처서(處暑)는 계절의 하나로, 보통 양력으로 8월 23일경에 해당하며, 더위가 한창 심한 시기가 끝나고 서서히 시원한 기운이 돌기 시작하는 시기입니다.',
 'source': '한국의 24절기 개념 및 날짜 기준'}